In [11]:
# Import necessary libraries and modules for your script
import glob  # For file path manipulation and searching
import torch  # PyTorch for deep learning
import torch.nn as nn  # Neural network components in PyTorch
from torch.utils.data import DataLoader  # DataLoader for handling datasets
from torch.optim import Adam  # Optimizer for training
from torch.autograd import Variable  # Variable for gradient computation
import torchvision  # Part of PyTorch for computer vision tasks

# Import user-defined modules for model utilities and components
import import_ipynb  # Import .ipynb notebooks as modules (if necessary)
from Model_Utils import classes, transform, transform_PIL, model  # Import model-related components

In [19]:
# Check for the availability of a CUDA-compatible GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device
# 'device' now holds the selected device for computation (either GPU or CPU)

device(type='cuda')

In [20]:
# Define the paths to the training, testing, and validation directories
train_path = 'Dataset/train'
test_path = 'Dataset/test'
valid_path = 'Dataset/valid'

# Create data loaders for the training, testing, and validation datasets
# Each data loader loads data from the specified directory, applies the 'transform',
# and shuffles the data in batches for efficient training and testing

# DataLoader for the training dataset
train_loader = DataLoader(
    torchvision.datasets.ImageFolder(train_path, transform=transform),
    batch_size=64, shuffle=True
)

# DataLoader for the testing dataset
test_loader = DataLoader(
    torchvision.datasets.ImageFolder(test_path, transform=transform),
    batch_size=32, shuffle=True
)

# DataLoader for the validation dataset
valid_loader = DataLoader(
    torchvision.datasets.ImageFolder(valid_path, transform=transform),
    batch_size=32, shuffle=True
)

In [21]:
# Move the model to the selected device (GPU or CPU)
model = model.to(device)

# Define the optimizer and loss function for training
optimizer = Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
loss_function = nn.CrossEntropyLoss()

In [22]:
# Calculate the number of images in the training, testing, and validation datasets
train_count = len(glob.glob(train_path + '/**/*.jpg'))
test_count = len(glob.glob(test_path + '/**/*.jpg'))
valid_count = len(glob.glob(valid_path + '/**/*.jpg'))

# Print the counts of images in each dataset
print("Number of training images:", train_count)
print("Number of testing images:", test_count)
print("Number of validation images:", valid_count)

Number of training images: 7988
Number of testing images: 233
Number of validation images: 436


Training without Cross Validation

In [25]:
# Initialize variables to track the best accuracy and the number of epochs
best_accuracy = 0.0
num_epochs = 15

# Loop through the specified number of epochs
for epoch in range(num_epochs):
    
    # Set the model in training mode
    model.train()
    train_accuracy = 0.0
    train_loss = 0.0
    
    # Training on the training dataset
    for i, (images, labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.cpu().data * images.size(0)
        _, prediction = torch.max(outputs.data, 1)
        
        train_accuracy += int(torch.sum(prediction == labels.data))
        
    train_accuracy = train_accuracy / len(train_loader.dataset)
    train_loss = train_loss / len(train_loader.dataset)
    
    # Set the model in evaluation mode
    model.eval()
    
    valid_accuracy = 0.0
    
    # Validation on the validation dataset
    for i, (images, labels) in enumerate(valid_loader):
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        outputs = model(images)
        _, prediction = torch.max(outputs.data, 1)
        valid_accuracy += int(torch.sum(prediction == labels.data))
    
    valid_accuracy = valid_accuracy / len(valid_loader.dataset)
    
    # Print training and validation statistics for the current epoch
    print('Epoch: ' + str(epoch) + ' Train Loss: ' + str(train_loss) + ' Train Accuracy: ' + str(train_accuracy) + ' Validation Accuracy: ' + str(valid_accuracy))
    
    # Save the best model based on validation accuracy
    if valid_accuracy > best_accuracy:
        torch.save(model.state_dict(), 'Best_Checkpoint.model')
        best_accuracy = valid_accuracy

# Load the best model for testing
model.load_state_dict(torch.load('Best_Checkpoint.model'))

# Set the model in evaluation mode
model.eval()

test_accuracy = 0.0

# Testing on the test dataset
for i, (images, labels) in enumerate(test_loader):
    if torch.cuda.is_available():
        images = Variable(images.cuda())
        labels = Variable(labels.cuda())
        
    outputs = model(images)
    _, prediction = torch.max(outputs.data, 1)
    test_accuracy += int(torch.sum(prediction == labels.data))

test_accuracy = test_accuracy / len(test_loader.dataset)

# Print the final test accuracy
print('Test Accuracy: ' + str(test_accuracy))


Epoch: 0 Train Loss: tensor(0.1849) Train Accuracy: 0.9201301952929394 Validation Accuracy: 0.5871559633027523


Training with Cross Validation

In [ ]:
# Define the number of epochs for each fold and the number of folds for cross-validation
num_epochs = 15
num_folds = 5

# Initialize a variable to track the best accuracy across all folds
best_accuracy = 0.0

# Loop through each fold for cross-validation
for fold in range(num_folds):
    print(f"Fold {fold + 1}/{num_folds}")
    
    # Split the dataset into training and validation sets for the current fold
    
    # Create a model and optimizer for each fold if needed
    
    for epoch in range(num_epochs):
        # Set the model in training mode
        model.train()
        train_accuracy = 0.0
        train_loss = 0.0

        # Training on the training dataset
        for i, (images, labels) in enumerate(train_loader):
            if torch.cuda.is_available():
                images = Variable(images.cuda())
                labels = Variable(labels.cuda())

            optimizer.zero_grad()

            outputs = model(images)
            loss = loss_function(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.cpu().data * images.size(0)
            _, prediction = torch.max(outputs.data, 1)

            train_accuracy += int(torch.sum(prediction == labels.data))

        train_accuracy = train_accuracy / len(train_loader.dataset)
        train_loss = train_loss / len(train_loader.dataset)

        # Set the model in evaluation mode
        model.eval()

        valid_accuracy = 0.0
        for i, (images, labels) in enumerate(valid_loader):
            if torch.cuda.is_available():
                images = Variable(images.cuda())
                labels = Variable(labels.cuda())

            outputs = model(images)
            _, prediction = torch.max(outputs.data, 1)
            valid_accuracy += int(torch.sum(prediction == labels.data))

        valid_accuracy = valid_accuracy / len(valid_loader.dataset)

        print(f'Fold {fold + 1}/{num_folds} - Epoch: {epoch + 1}, Train Loss: {train_loss}, Train Accuracy: {train_accuracy}, Validation Accuracy: {valid_accuracy}')

        # Save the best model based on validation accuracy
        if valid_accuracy > best_accuracy:
            torch.save(model.state_dict(), f'Best_Checkpoint_Cross.model')
            best_accuracy = valid_accuracy

# Load the best model for testing
model.load_state_dict(torch.load(f'Best_Checkpoint_Cross.model'))

# Set the model in evaluation mode
model.eval()

test_accuracy = 0.0

# Testing on the test dataset
for i, (images, labels) in enumerate(test_loader):
    if torch.cuda.is_available():
        images = Variable(images.cuda())
        labels = Variable(labels.cuda())

    outputs = model(images)
    _, prediction = torch.max(outputs.data, 1)
    test_accuracy += int(torch.sum(prediction == labels.data))

test_accuracy = test_accuracy / len(test_loader.dataset)

# Print the final test accuracy
print(f'Test Accuracy: {test_accuracy}')

Fold 1/5
Fold 1/5 - Epoch: 1, Train Loss: 14.983320236206055, Train Accuracy: 0.4046303604314654, Validation Accuracy: 0.49842767295597484
Fold 1/5 - Epoch: 2, Train Loss: 1.3507047891616821, Train Accuracy: 0.6617556783302639, Validation Accuracy: 0.5833333333333334
Fold 1/5 - Epoch: 3, Train Loss: 0.6818678975105286, Train Accuracy: 0.8046128211874068, Validation Accuracy: 0.6446540880503144
Fold 1/5 - Epoch: 4, Train Loss: 0.48063430190086365, Train Accuracy: 0.876786810488468, Validation Accuracy: 0.5927672955974843
Fold 1/5 - Epoch: 5, Train Loss: 0.40458497405052185, Train Accuracy: 0.9093221082171359, Validation Accuracy: 0.6713836477987422
Fold 1/5 - Epoch: 6, Train Loss: 0.36300745606422424, Train Accuracy: 0.9241427694466369, Validation Accuracy: 0.6698113207547169
Fold 1/5 - Epoch: 7, Train Loss: 0.3234556019306183, Train Accuracy: 0.9344909234411997, Validation Accuracy: 0.6792452830188679
Fold 1/5 - Epoch: 8, Train Loss: 0.311686635017395, Train Accuracy: 0.933087783916513